In [ ]:
import qspectra as qs
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
H_electronic = np.array([[11500, 66], [66, 11650]])
n_vibrational_levels = np.array([2, 2])
vib_energies = np.array([200, 200])
elec_vib_couplings = -32 * np.eye(2)
bath = qs.DebyeBath(CM_K * 77, 1.5 * 35, 106)
dipoles = [[1,0,0], [0,1,0]]

dimer_ham = qs.VibronicHamiltonian(
    qs.ElectronicHamiltonian(H_electronic, dipoles=dipoles, bath=bath),
    n_vibrational_levels, vib_energies, elec_vib_couplings)
dimer = qs.RedfieldModel(dimer_ham, hilbert_subspace='gef', unit_convert=qs.CM_FS,
                      discard_imag_corr=True)

## Excited state dynamics

In [ ]:
%%time 
t, rho = qs.simulate_dynamics(dimer, unit_vec(0, 2 * 2 ** 2), 1000)

In [ ]:
plt.plot(t, np.einsum('tii->ti', rho.reshape(-1, 8, 8)).real);

## Absorption spectra

In [ ]:
%%time
f, X = qs.absorption_spectra(dimer, 10000)

In [ ]:
plt.plot(f, X)
plt.xlim(11000, 12000);

## Monomer absorption spectra (for reference)

In [ ]:
monomer_ham = qs.VibronicHamiltonian(
    qs.ElectronicHamiltonian(H_electronic[:1, :1], dipoles=dipoles[:1], bath=bath),
    n_vibrational_levels[:1], vib_energies[:1], elec_vib_couplings[:1, :1])
monomer = qs.RedfieldModel(monomer_ham, hilbert_subspace='ge', unit_convert=qs.CM_FS)

In [ ]:
%%time
f, X = qs.absorption_spectra(monomer, 10000)

In [ ]:
plt.plot(f, X)
plt.xlim(11000, 12000);

## 2D spectra

In [ ]:
%%time
(t1, t2, t3), X = qs.third_order_response(dimer, coherence_time_max=1000,
                                          population_times=np.linspace(0, 1000, num=100),
                                          geometry='-++', polarization='xxxx',
                                          include_signal='GSB', exact_isotropic_average=True)

In [ ]:
%%time
f1, X_ftt = qs.fourier_transform(t1, X, axis=0, rw_freq=dimer.rw_freq, sign=-1, unit_convert=qs.CM_FS)
f3, X_ftf = qs.fourier_transform(t3, X_ftt, axis=2, rw_freq=dimer.rw_freq, unit_convert=qs.CM_FS)

In [ ]:
plt.figure(figsize=(6,5))
plt.contourf(f3, f1, X_ftf[:,0,:].real.T, 50)
plt.xlim(11100, 12000)
plt.ylim(11100, 12000)
plt.colorbar();

In [ ]:
alpha, beta = np.argmin(np.abs(f1 - np.array([11530, 11730]).reshape(-1,1)), 1)

In [ ]:
plt.plot(t2, X_ftf[beta, :, alpha].real, '-sr')
plt.plot(t2, X_ftf[alpha, :, beta].real, '-sg');